In [ ]:
import pandas as pd
import requests
import warnings

# Ignore future change warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Replace API_KEY with your own EIA API key
API_KEY = "o4arSd634GN4GgFsgUwj5x4f8mG8hNDt7L6ixIs3"

# Specify the API endpoint and parameters
url = "https://api.eia.gov/v2/electricity/rto/daily-fuel-type-data/data/"
params = {
    "api_key": API_KEY,
    "frequency": "daily",
    "data[0]": "value",
    "start": "2017-01", 
    "end": "2018-12",
    "sort[0][column]": "period",
    "sort[0][direction]": "desc",
    "offset": "0",
    "length": "5000"
}

# Send a GET request to the API endpoint and fetch the response
response = requests.get(url, params=params)

# Check if the response contains valid JSON data
try:
    data = response.json()
except json.JSONDecodeError as e:
    print("Error decoding JSON:", e)
    print("Response content:", response.content)
    data = None

if data:
    # Check if the 'response' key is present
    if 'response' in data:
        # Convert the response to a Pandas DataFrame
        df = pd.DataFrame(data['response']['data'])

        # Check if there are more pages of data to fetch
        while len(data['response']['data']) == int(params['length']):
            # Update the offset parameter to fetch the next page of data
            params['offset'] = str(int(params['offset']) + int(params['length']))
            
            # Send a GET request to the API endpoint with the updated parameters and fetch the response
            response = requests.get(url, params=params)
            
            # Check if the response contains valid JSON data
            try:
                data = response.json()
            except json.JSONDecodeError as e:
                print("Error decoding JSON:", e)
                print("Response content:", response.content)
                break
            
            # Check if the 'response' key is present in the new data
            if 'response' in data:
                # Convert the response to a Pandas DataFrame and append to the existing DataFrame
                df = df.append(pd.DataFrame(data['response']['data']), ignore_index=True)
            else:
                print("Invalid response structure. 'response' key not found.")
                break

        # Save the DataFrame to a CSV file
        df.to_csv("day_ahead_demand_forecast5.csv", index=False)
    else:
        print("Invalid response structure. 'response' key not found.")
